# Trabajo Práctico N° 1

**Año:** 2024  

**Materia:** Aprendizaje Automatico 1

**Integrantes:** Avecilla Tomás, Calcia Franco



### Diccionario  de Datos

| Columna       | Descripción                                                                                          |
|---------------|---------------------------------------------------------------------------------------------------------------|
| **CRIM** | Tasa de criminalidad per cápita por ciudad.                       |
| **ZN**   | Proporción de terrenos residenciales zonificados para lotes de más de 25,000 pies cuadrados.                         |
| **INDUS**   | Proporción de acres de negocios no minoristas por ciudad.                         |
| **CHAS** | Variable dummy del río Charles (1 si el tramo limita con el río; 0 de lo contrario).                                                             |
| **NOX**   | Concentración de óxidos de nitrógeno (partes por 10 millones) [parts/10M].                                                          |
| **RM**        | Número promedio de habitaciones por vivienda.|
| **AGE**    | Proporción de unidades ocupadas por sus propietarios construidas antes de 1940.                                                        |
| **DIS**   | Distancias ponderadas a cinco centros de empleo de Boston.       |
| **RAD** | índice de accesibilidad a las autopistas radiales.                                                             |
| **TAX**   | Tasa de impuesto sobre la propiedad a valor completo por $10,000 [$/10k].                                                          |
| **PTRATIO**        | Proporción alumno-maestro por ciudad.|
| **B**    | El resultado de la ecuación B=1000(Bk - 0.63)^2 donde Bk es la proporción de negros por ciudad.                                                        |
| **LSTAT**   | % de población de menor estatus socioeconómico.       |
| **MEDV** *(Variable de salida)*  | Valor mediano de las viviendas ocupadas por sus propietarios en miles de dólares [k$].   | 


# Preparacion del entorno de Trabajo

In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import scipy.stats as stats
import statsmodels.api as sm
from sklearn.impute import KNNImputer
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LinearRegression, LassoCV, RidgeCV, ElasticNetCV
from sklearn.metrics import mean_squared_error, r2_score, mean_absolute_percentage_error, mean_absolute_error
from matplotlib.colors import LinearSegmentedColormap
from sklearn.preprocessing import RobustScaler


: 

In [35]:
df_precios_casas = pd.read_csv("house-prices-tp.csv")

# Train-Test
Decidimos hacer la division del dataset antes de comenzar el analisis y hacer cualquier tratado de datos ya que necesitamos que el dataset de testeo sea tomado como datos desconocidos

In [36]:
X_train, X_test, y_train, y_test = train_test_split(df_precios_casas.drop(columns=['MEDV']), df_precios_casas['MEDV'], test_size=0.2, random_state=42)

df_entrenamiento = pd.concat([X_train,y_train],axis=1)
df_test = pd.concat([X_test,y_test],axis=1)

# Análisis Exploratorio

## Limpieza de Datos

### Verificamos valores nulos

In [ ]:
conteo_nan = df_entrenamiento.isnull().sum()
conteo_nan

Como primera medida eliminaremos las fila que tengan la variable de salida nula o mas de 11 columnas nulas ya que las consideramos irrelevantes para el analisis

In [ ]:
filas_nan = []

for index, row in df_entrenamiento.iterrows():
    # Verificamos si la fila tiene más de 11 NaN o si el valor de MEDV es NaN
    if row.isnull().sum() > 11 or pd.isnull(row['MEDV']):
        filas_nan.append(index)

df_entrenamiento.drop(index=filas_nan, inplace=True)

print(f"Filas eliminadas: {len(filas_nan)}")


In [ ]:
conteo_nan = df_entrenamiento.isnull().sum()
conteo_nan

Imputaremos la columna binaria con la moda

In [40]:
df_entrenamiento['CHAS'] = df_entrenamiento['CHAS'].fillna(df_entrenamiento['CHAS'].mode()[0])

Y el resto de columnas seran imputadas con KNN

In [41]:
imputer = KNNImputer(n_neighbors=5)
df_entrenamiento = pd.DataFrame(imputer.fit_transform(df_entrenamiento), columns=df_entrenamiento.columns)

In [ ]:
conteo_nan = df_entrenamiento.isnull().sum()
conteo_nan

### Exploracion de outliers

In [ ]:
sns.set_theme(style="whitegrid")

num_columns = len(df_entrenamiento.columns)

max_plots_per_figure = 12
num_figures = (num_columns + max_plots_per_figure - 1) // max_plots_per_figure

for fig_index in range(num_figures):
    start_index = fig_index * max_plots_per_figure
    end_index = min(start_index + max_plots_per_figure, num_columns)

    plt.figure(figsize=(15, 5 * ((end_index - start_index + 2) // 2)))

    for i, column in enumerate(df_entrenamiento.columns[start_index:end_index]):
        plt.subplot((end_index - start_index + 2) // 3, 3, i + 1)
        if column == 'CHAS':
            sns.countplot(data=df_entrenamiento, x='CHAS')
            plt.title('Distribución de CHAS')
        else:
            sns.boxplot(data=df_entrenamiento[column], color="skyblue")
            plt.title(column)
        plt.xlabel('')

    plt.tight_layout()

    plt.show()


Podemos notar varias variables con presencia de outliers por lo cual deberiamos hacer un analisis mas profundo

In [ ]:
resultados_atipicos = {}

for columna in df_entrenamiento.columns:
    if df_entrenamiento[columna].dtype in ['int64', 'float64']:
        Q1 = df_entrenamiento[columna].quantile(0.25)
        Q3 = df_entrenamiento[columna].quantile(0.75)
        IQR = Q3 - Q1

        limite_inferior = Q1 - 1.5 * IQR
        limite_superior = Q3 + 1.5 * IQR

        es_atipico = (df_entrenamiento[columna] < limite_inferior) | (df_entrenamiento[columna] > limite_superior)

        cantidad_atipicos = es_atipico.sum()
        porcentaje_atipicos = (cantidad_atipicos / len(df_entrenamiento)) * 100

        resultados_atipicos[columna] = porcentaje_atipicos

for columna, porcentaje in resultados_atipicos.items():
    print(f"Columna: {columna} - Porcentaje de valores atípicos: {porcentaje:.2f}%")

## Analisis Descriptivo

In [ ]:
df_entrenamiento.describe()

In [ ]:
df_entrenamiento.info()

In [ ]:
plt.figure(figsize=(10, 6))
sns.histplot(df_entrenamiento['MEDV'], bins=30, kde=True)
plt.title('variable Target')
plt.xlabel('Precio')
plt.ylabel('Frecuencia')
plt.show()

In [ ]:
sns.set_theme(style="whitegrid")

sns.pairplot(
    df_entrenamiento,
    diag_kind='kde',
    plot_kws={'alpha': 0.6, 'color': 'orange', 's': 15},
    diag_kws={'color': 'red', 'fill': True},  # Cambiado 'shade' por 'fill'
    height=2.5,
    aspect=1,
    corner=True,
)

plt.suptitle('Pairplot de Variables del DataFrame de Entrenamiento', fontsize=16, fontweight='bold', y=1.02)

plt.show()


#### Matriz de correlación

In [ ]:
corr_matrix = df_entrenamiento.corr()

mask = np.triu(np.ones_like(corr_matrix, dtype=bool))

colors = [(1, 1, 1), (1, 0, 0)]
n_bins = 100
cmap_name = 'red_white'
cm = LinearSegmentedColormap.from_list(cmap_name, colors, N=n_bins)

plt.figure(figsize=(12, 8))
sns.heatmap(corr_matrix, mask=mask, annot=True, cmap='vlag', center=0, vmin=-1, vmax=1, linewidths=0.5)
plt.title('Matriz de correlación')
plt.show()

Notamos una fuerte relacion entre las variables TAX-RAD (0.88) y NOX-INDUS (0.74)

# Valores nulos en dataset de testeo

In [ ]:
df_test.isnull().sum()

Usamos el mismo criterio que en el dataset de entrenamiento para eliminar filas completas

In [ ]:
filas_nan = []

for index, row in df_test.iterrows():
    # Verificamos si la fila tiene más de 11 NaN o si el valor de MEDV es NaN
    if row.isnull().sum() > 11 or pd.isnull(row['MEDV']):
        filas_nan.append(index)

df_test.drop(index=filas_nan, inplace=True)

print(f"Filas eliminadas: {len(filas_nan)}")

Aqui repetimos el criterio de rellenar con la moda que en el caso del entrenamiento fue de `0`

In [52]:
df_test['CHAS'] = df_test['CHAS'].fillna(0)

In [ ]:
df_test.isnull().sum()

Y por ultimo usamos el mismo `imputer` para rellenar los restantes

In [54]:
df_test = pd.DataFrame(imputer.fit_transform(df_test), columns=df_test.columns)

In [ ]:
df_test.isnull().sum()

------------------------------------------------------------------------------------------------------

# Escalar los datos

In [56]:
# Separar en X_train, y_train, X_test, y_test
X_train = df_entrenamiento.drop(columns=['MEDV'])
y_train = df_entrenamiento['MEDV']
X_test = df_test.drop(columns=['MEDV'])
y_test = df_test['MEDV']

In [57]:
# Escalar los datos
scaler = RobustScaler()
X_train_scaled = scaler.fit_transform(X_train)
X_test_scaled = scaler.transform(X_test)


# Regresión lineal simple

In [ ]:
regresion_lineal = LinearRegression()
regresion_lineal.fit(X_train_scaled, y_train)

# Gradientes Descendentes

## Gradiente Descendente (Batch)

In [60]:
def gradient_descent(X_train, y_train, X_val, y_val, lr, epochs):
    """
    shapes:
        X_train = nxm
        y_train = nx1
        X_val = pxm
        y_val = px1
        W = mx1
    """
    n = X_train.shape[0]  # Número de ejemplos de entrenamiento
    m = X_train.shape[1]  # Número de características

    o = X_val.shape[0]  # Número de ejemplos de validación

    # Poner columna de unos a las matrices X para el término de sesgo (bias)
    X_train = np.hstack((np.ones((n, 1)), X_train))
    X_val = np.hstack((np.ones((o, 1)), X_val))

    # Convertir y_train y y_val a arrays de NumPy y asegurarse de que sean vectores columna
    y_train = y_train.to_numpy().reshape(-1, 1)
    y_val = y_val.to_numpy().reshape(-1, 1)

    # Inicializar pesos aleatorios
    W = np.random.randn(m + 1).reshape(m + 1, 1)

    train_errors = []  # Para almacenar el error de entrenamiento en cada época
    val_errors = []    # Para almacenar el error de validación en cada época

    for _ in range(epochs):
        # Calcular predicción y error de entrenamiento
        prediction_train = np.matmul(X_train, W)
        error_train = y_train - prediction_train
        train_mse = np.mean(error_train ** 2)
        train_errors.append(train_mse)

        # Calcular predicción y error de validación
        prediction_val = np.matmul(X_val, W)
        error_val = y_val - prediction_val
        val_mse = np.mean(error_val ** 2)
        val_errors.append(val_mse)

        # Calcular el gradiente y actualizar pesos
        grad_sum = np.sum(error_train * X_train, axis=0)
        grad_mul = -2 / n * grad_sum  # 1xm
        gradient = np.transpose(grad_mul).reshape(-1, 1)  # mx1

        W = W - (lr * gradient)

    # Graficar errores de entrenamiento y validación
    plt.figure(figsize=(12, 6))
    plt.plot(train_errors, label='Error de entrenamiento')
    plt.plot(val_errors, label='Error de validación')
    plt.xlabel('Época')
    plt.ylabel('Error cuadrático medio')
    plt.legend()
    plt.title('Error de entrenamiento y validación vs iteraciones (GD)')
    plt.show()

    return W

## Gradiente Descendente Estocastico (SGD)

In [59]:
def stochastic_gradient_descent(X_train, y_train, X_test, y_test, lr, epochs):
    n = X_train.shape[0]
    m = X_train.shape[1]

    # Agregar el término de sesgo (bias) a las matrices de entrada
    X_train = np.hstack((np.ones((n, 1)), X_train))
    X_test = np.hstack((np.ones((X_test.shape[0], 1)), X_test))

    # Inicializar los pesos de forma aleatoria
    W = np.random.randn(m + 1, 1)

    train_errors = []
    test_errors = []

    # Convertir y_train y y_test a numpy.ndarray
    y_train = y_train.to_numpy().reshape(-1, 1)
    y_test = y_test.to_numpy().reshape(-1, 1)

    for i in range(epochs):
        # Permutación aleatoria de los datos
        permutation = np.random.permutation(n)
        X_train = X_train[permutation]
        y_train = y_train[permutation].reshape(-1, 1)  # Asegurarse de que y_train sea un vector columna

        for j in range(n):
            # Obtener una muestra aleatoria de un solo dato para hacer SGD
            x_sample = X_train[j].reshape(1, -1)
            y_sample = y_train[j]

            prediction = np.matmul(x_sample, W)
            error = y_sample - prediction
            train_mse = error ** 2
            train_errors.append(train_mse[0])

            prediction_test = np.matmul(X_test, W)
            error_test = y_test - prediction_test  # Asegurarse de que y_test sea un vector columna
            test_mse = np.mean(error_test ** 2)
            test_errors.append(test_mse)

            gradient = -2 * error * x_sample.T
            W = W - (lr * gradient)

            
    plt.figure(figsize=(12, 6))
    plt.plot(train_errors, label='Error de entrenamiento')
    plt.plot(test_errors, label='Error de prueba')
    plt.xlabel('Iteración')
    plt.ylabel('Error cuadrático medio')
    plt.legend()
    plt.title('Error de entrenamiento y prueba vs iteraciones (SGD)')
    plt.show()

    return W

## Gradiente Descedente por lotes (Mini-Batch)

In [61]:
def mini_batch_gradient_descent(X_train, y_train, X_test, y_test, lr, epochs, batch_size):
    n = X_train.shape[0]
    m = X_train.shape[1]

    X_train = np.hstack((np.ones((n, 1)), X_train))
    X_test = np.hstack((np.ones((X_test.shape[0], 1)), X_test))

    y_train = y_train.to_numpy().reshape(-1, 1)
    y_test = y_test.to_numpy().reshape(-1, 1)

    W = np.random.randn(m + 1).reshape(-1, 1)

    train_errors = []
    test_errors = []

    for i in range(epochs):
        # Permutación aleatoria de los datos
        permutation = np.random.permutation(n)
        X_train = X_train[permutation]
        y_train = y_train[permutation]

        for j in range(0, n, batch_size):
            # Obtener un lote (mini-batch) de datos
            x_batch = X_train[j:j+batch_size, :]
            y_batch = y_train[j:j+batch_size].reshape(-1, 1)

            prediction = np.matmul(x_batch, W)
            error = y_batch - prediction
            train_mse = np.mean(error ** 2)
            train_errors.append(train_mse)

            gradient = -2 * np.matmul(x_batch.T, error) / batch_size

            W = W - (lr * gradient)

            prediction_test = np.matmul(X_test, W)
            error_test = y_test - prediction_test
            test_mse = np.mean(error_test ** 2)
            test_errors.append(test_mse)

            
    plt.figure(figsize=(12, 6))
    plt.plot(train_errors, label='Error de entrenamiento')
    plt.plot(test_errors, label='Error de prueba')
    plt.xlabel('Iteración')
    plt.ylabel('Error cuadrático medio')
    plt.legend()
    plt.title('Error de entrenamiento y prueba vs iteraciones (Mini-Batch GD)')
    plt.show()

    return W

# Modelos de Regularización

In [62]:
regresiones_lineales = {
    "Lasso": LassoCV(alphas=np.logspace(-4, 4, 20), cv=5, random_state=42),
    "Ridge": RidgeCV(alphas=np.logspace(-4, 4, 20), cv=5),
    "ElasticNet": ElasticNetCV(alphas=np.logspace(-4, 4, 20), l1_ratio=np.linspace(0.1, 1, 10), cv=5, random_state=42)
}

# Evaluación de Modelos

In [63]:
def evaluar_modelo(modelo, nombre_modelo, X_train, y_train, X_test, y_test):
    y_train_pred = modelo.predict(X_train)
    y_test_pred = modelo.predict(X_test)

    resultados = {
        "Modelo": nombre_modelo,
        "Train RMSE": np.sqrt(mean_squared_error(y_train, y_train_pred)),
        "Test RMSE": np.sqrt(mean_squared_error(y_test, y_test_pred)),
        "Train R2": r2_score(y_train, y_train_pred),
        "Test R2": r2_score(y_test, y_test_pred),
        "Train MSE": mean_squared_error(y_train, y_train_pred),
        "Test MSE": mean_squared_error(y_test, y_test_pred),
        "Train MAE": mean_absolute_error(y_train, y_train_pred),
        "Test MAE": mean_absolute_error(y_test, y_test_pred),
        "Train MAPE": mean_absolute_percentage_error(y_train, y_train_pred),
        "Test MAPE": mean_absolute_percentage_error(y_test, y_test_pred)
    }

    return resultados

def calcular_metricas(y_true, y_pred):
    mse = mean_squared_error(y_true, y_pred)
    mae = mean_absolute_error(y_true, y_pred)
    mape = mean_absolute_percentage_error(y_true, y_pred)
    return mse, mae, mape

def evaluar_gradiente(W, X_train, X_test, y_train, y_test, nombre_modelo):
    X_train_bias = np.hstack((np.ones((X_train.shape[0], 1)), X_train))
    X_test_bias = np.hstack((np.ones((X_test.shape[0], 1)), X_test))

    y_train_pred = np.dot(X_train_bias, W).flatten()
    y_test_pred = np.dot(X_test_bias, W).flatten()

    train_mse, train_mae, train_mape = calcular_metricas(y_train, y_train_pred)
    test_mse, test_mae, test_mape = calcular_metricas(y_test, y_test_pred)

    resultados = {
        "Modelo": nombre_modelo,
        "Train RMSE": np.sqrt(train_mse),
        "Test RMSE": np.sqrt(test_mse),
        "Train R2": r2_score(y_train, y_train_pred),
        "Test R2": r2_score(y_test, y_test_pred),
        "Train MSE": train_mse,
        "Test MSE": test_mse,
        "Train MAE": train_mae,
        "Test MAE": test_mae,
        "Train MAPE": train_mape,
        "Test MAPE": test_mape
    }

    return resultados

In [ ]:
resultados_finales = []

# Regresión lineal simple
resultados_finales.append(evaluar_modelo(regresion_lineal, "Regresión Lineal", X_train_scaled, y_train, X_test_scaled, y_test))

# Gradiente descendente (modelos obtenidos por SGD, Batch y Mini-batch)
W_sgd = stochastic_gradient_descent(X_train_scaled, y_train, X_test_scaled, y_test, lr=0.001, epochs=1000)
W_batch = gradient_descent(X_train_scaled, y_train, X_test_scaled, y_test, lr=0.001, epochs=1000)
W_mini_batch = mini_batch_gradient_descent(X_train_scaled, y_train, X_test_scaled, y_test, lr=0.01, epochs=100, batch_size=25)

# Evaluar los modelos obtenidos por gradientes
resultados_finales.append(evaluar_gradiente(W_sgd, X_train_scaled, X_test_scaled, y_train, y_test, "SGD"))
resultados_finales.append(evaluar_gradiente(W_batch, X_train_scaled, X_test_scaled, y_train, y_test, "Batch Gradient Descent"))
resultados_finales.append(evaluar_gradiente(W_mini_batch, X_train_scaled, X_test_scaled, y_train, y_test, "Mini-Batch Gradient Descent"))
# Evaluación de regularización (Lasso, Ridge, ElasticNet)
for name, regressor in regresiones_lineales.items():
    regressor.fit(X_train_scaled, y_train)
    resultados_finales.append(evaluar_modelo(regressor, name, X_train_scaled, y_train, X_test_scaled, y_test))

# Mostrar los resultados finales
resultados_df = pd.DataFrame(resultados_finales)
resultados_df.head(10)



# Interpretación de los Gráficos

## 1. Stochastic Gradient Descent (SGD)

**Análisis:**
- El error de entrenamiento es extremadamente errático, con oscilaciones muy grandes. Esto es esperado en SGD, ya que utiliza una muestra aleatoria de los datos en cada iteración, lo que genera una alta variabilidad.
- El error de prueba, por otro lado, es mucho más estable y más bajo, lo que sugiere que, a pesar de las oscilaciones del entrenamiento, el modelo generaliza bien en el conjunto de prueba.

**Conclusión:** Aunque SGD tiene un comportamiento muy volátil en el entrenamiento, parece estar generalizando bien en el conjunto de prueba. Sin embargo, esta volatilidad podría reducirse ajustando la tasa de aprendizaje o implementando técnicas como el decaimiento de la tasa de aprendizaje.

## 2. Gradient Descent (GD)

**Análisis:**
- El error disminuye rápidamente al principio y luego se estabiliza alrededor de una misma línea para el entrenamiento y la validación, lo que indica que el modelo está convergiendo de manera efectiva.
- No hay oscilaciones pronunciadas, lo cual es característico del Gradient Descent estándar, ya que utiliza todo el conjunto de datos para cada iteración.

**Conclusión:** GD muestra un comportamiento de convergencia suave y estable, con un bajo riesgo de sobreajuste, ya que los errores de entrenamiento y validación son casi idénticos.

## 3. Mini-Batch Gradient Descent (MBGD)

**Análisis:**
- El error de entrenamiento disminuye rápidamente en las primeras iteraciones, lo cual es un comportamiento esperado.
- Se observa que oscila considerablemente a lo largo de las iteraciones, lo que es típico en el Mini-Batch GD debido a la naturaleza de los lotes pequeños de datos.
- El error de prueba sigue un patrón similar pero tiene menos variabilidad, lo que sugiere que el modelo es estable en el conjunto de prueba a pesar de las oscilaciones en el entrenamiento.

**Conclusión:** Mini-Batch GD logra una rápida convergencia inicial, pero las oscilaciones indican que podría necesitar una tasa de aprendizaje más pequeña o un tamaño de lote más grande para reducir la variabilidad.

# Interpretación del Cuadro

## Regresión Lineal:
- **Train RMSE (5.66) vs. Test RMSE (6.36):** El error en el conjunto de prueba es ligeramente mayor que en el conjunto de entrenamiento, lo que puede indicar un buen ajuste, aunque podría haber margen de mejora en la generalización.
- **Train R² (0.63) vs. Test R² (0.58):** El modelo explica el 63% de la variación en el conjunto de entrenamiento y el 58% en el conjunto de prueba, lo que muestra una ligera disminución en la capacidad predictiva cuando se prueban datos nuevos.
- **Train MSE (32.07) vs. Test MSE (40.45):** El error cuadrático medio es considerablemente mayor en el conjunto de prueba, reflejando la misma tendencia que el RMSE.
- **Train MAE (3.81) vs. Test MAE (4.22):** El error absoluto medio aumenta en el conjunto de prueba, lo que indica que el modelo es menos preciso con datos nuevos.
- **Train MAPE (0.19) vs. Test MAPE (0.23):** El error porcentual absoluto medio también incrementa en el conjunto de prueba, sugiriendo una disminución en la precisión predictiva.

**Conclusión:** El modelo de regresión lineal presenta un desempeño moderado, con un aumento en los errores al pasar al conjunto de prueba, lo que podría señalar sobreajuste.

## SGD (Stochastic Gradient Descent):
- **Train RMSE (5.68) vs. Test RMSE (6.46):** El error en el conjunto de prueba es un poco mayor que en el entrenamiento, lo que es normal y sugiere una buena generalización.
- **Train R² (0.65) vs. Test R² (0.34):** La capacidad predictiva disminuye considerablemente en el conjunto de prueba, lo que indica que el modelo no se ajusta bien a datos no vistos.
- **Train MSE (32.21) vs. Test MSE (41.78):** El error cuadrático medio sigue la misma tendencia que el RMSE, con un aumento en el conjunto de prueba.
- **Train MAE (3.78) vs. Test MAE (4.21):** El error absoluto medio es bajo en ambos conjuntos.
- **Train MAPE (0.18) vs. Test MAPE (0.23):** El error porcentual absoluto medio es consistente, aunque con una ligera pérdida de precisión en el conjunto de prueba.

**Conclusión:** Aunque el modelo SGD generaliza bien, la gran diferencia en el R² entre los conjuntos de entrenamiento y prueba sugiere que podría necesitar ajustes adicionales para mejorar la precisión en datos nuevos.

## Batch Gradient Descent:
- **Train RMSE (7.79) vs. Test RMSE (7.59):** El error en el conjunto de prueba es ligeramente menor que en el entrenamiento, lo que puede indicar que el modelo está subajustado.
- **Train R² (0.29) vs. Test R² (0.39):** El modelo tiene un mejor desempeño en el conjunto de prueba que en el entrenamiento, lo cual es indicativo de subajuste.
- **Train MSE (63.23) vs. Test MSE (57.59):** El error cuadrático medio es mayor en el conjunto de entrenamiento, lo que sigue la misma tendencia que el RMSE.
- **Train MAE (6.12) vs. Test MAE (6.18):** El error absoluto medio es muy similar en ambos conjuntos.
- **Train MAPE (0.28) vs. Test MAPE (0.31):** El error porcentual absoluto medio es moderadamente mayor en el conjunto de prueba.

**Conclusión:** El modelo Batch Gradient Descent parece estar subajustado, ya que su desempeño en el conjunto de prueba es mejor que en el conjunto de entrenamiento.

## Mini-Batch Gradient Descent:
- **Train RMSE (5.78) vs. Test RMSE (6.49):** El error aumenta en el conjunto de prueba, pero sigue siendo relativamente bajo, lo que indica que el modelo generaliza bien.
- **Train R² (0.62) vs. Test R² (0.56):** La capacidad predictiva del modelo se mantiene fuerte, aunque disminuye ligeramente en el conjunto de prueba.
- **Train MSE (33.15) vs. Test MSE (42.21):** El error cuadrático medio aumenta en el conjunto de prueba, siguiendo la tendencia del RMSE.
- **Train MAE (4.11) vs. Test MAE (4.34):** El error absoluto medio es bajo en ambos conjuntos, aunque ligeramente mayor en el conjunto de prueba.
- **Train MAPE (0.21) vs. Test MAPE (0.25):** El error porcentual absoluto medio es bajo, pero aumenta ligeramente en el conjunto de prueba.

**Conclusión:** El modelo Mini-Batch Gradient Descent ofrece un buen balance entre ajuste y generalización, con errores relativamente bajos y consistentes.

## Lasso:
- **Train RMSE (5.66) vs. Test RMSE (6.63):** El error en el conjunto de prueba es mayor que en el entrenamiento, similar al comportamiento de la regresión lineal.
- **Train R² (0.63) vs. Test R² (0.57):** La capacidad predictiva es similar a la regresión lineal, con una ligera disminución en el conjunto de prueba.
- **Train MSE (32.08) vs. Test MSE (40.41):** El error cuadrático medio refleja la misma tendencia que el RMSE.
- **Train MAE (3.80) vs. Test MAE (4.19):** El error absoluto medio es bajo, aunque aumenta en el conjunto de prueba.
- **Train MAPE (0.19) vs. Test MAPE (0.23):** El error porcentual absoluto medio también aumenta en el conjunto de prueba.

**Conclusión:** Lasso no ofrece mejoras significativas en comparación con la regresión lineal, indicando que la regularización L1 no es efectiva en este caso.

## Ridge:
- **Train RMSE (5.70) vs. Test RMSE (6.36):** El error es alto en ambos conjuntos y sigue una tendencia similar a los modelos lineales.
- **Train R² (0.63) vs. Test R² (0.58):** La capacidad predictiva es similar a los modelos anteriores.
- **Train MSE (32.50) vs. Test MSE (40.45):** Los errores cuadráticos medios son consistentes con el RMSE.
- **Train MAE (3.80) vs. Test MAE (4.16):** Los errores absolutos medios son similares en ambos conjuntos.
- **Train MAPE (0.19) vs. Test MAPE (0.23):** El error porcentual absoluto medio es consistente con los otros modelos.

**Conclusión:** Ridge no mejora significativamente el rendimiento con respecto a los modelos previos, lo que sugiere que la regularización L2 no es efectiva en este contexto.

## ElasticNet:
- **Train RMSE (5.70) vs. Test RMSE (6.36):** Los errores son casi idénticos a los de los modelos de regularización Lasso y Ridge.
- **Train R² (0.63) vs. Test R² (0.58):** Capacidad predictiva similar a los demás modelos.
- **Train MSE (32.57) vs. Test MSE (40.45):** Los errores cuadráticos medios son consistentes con el RMSE.
- **Train MAE (3.80) vs. Test MAE (4.16):** El error absoluto medio es alto en ambos conjuntos.
- **Train MAPE (0.19) vs. Test MAPE (0.23):** El error porcentual absoluto medio es consistente con los otros modelos.

**Conclusión:** ElasticNet no mejora significativamente respecto a Lasso o Ridge, lo que sugiere que la combinación de penalizaciones L1 y L2 no aporta beneficios en este caso.

---

# Conclusión parcial:
- **Modelos de Regularización (Lasso, Ridge, ElasticNet) y Regresión Lineal:**
  - Presentan errores similares.
  - La regularización no mejora significativamente el rendimiento.

- **Modelos Basados en Gradiente:**
  - **Mini-Batch Gradient Descent** y **SGD** muestran un rendimiento superior.
  - **Batch Gradient Descent** parece estar subajustado.

- **General:**
  - Los métodos de gradiente, especialmente **Mini-Batch Gradient Descent**, logran un buen equilibrio entre ajuste y generalización.

# Optimización de los hiperparámetros.

In [65]:
regresiones_lineales = {
    "Lasso": LassoCV(alphas=np.logspace(-6, 2, 50), cv=5, max_iter=10000),
    "Ridge": RidgeCV(alphas=np.logspace(-6, 2, 50), cv=5),
    "ElasticNet": ElasticNetCV(alphas=np.logspace(-4, 4, 20), l1_ratio=np.linspace(0.1, 1, 10), cv=5, random_state=42)
}

In [ ]:
resultados_finales = []

# Regresión lineal simple
resultados_finales.append(evaluar_modelo(regresion_lineal, "Regresión Lineal", X_train_scaled, y_train, X_test_scaled, y_test))

# Gradiente descendente (modelos obtenidos por SGD, Batch y Mini-batch)
W_sgd = stochastic_gradient_descent(X_train_scaled, y_train, X_test_scaled, y_test, lr=0.001, epochs=15)
W_batch = gradient_descent(X_train_scaled, y_train, X_test_scaled, y_test, lr=0.01, epochs=650)
W_mini_batch = mini_batch_gradient_descent(X_train_scaled, y_train, X_test_scaled, y_test, lr=0.01, epochs=1120, batch_size=240)

# Evaluar los modelos obtenidos por gradientes
resultados_finales.append(evaluar_gradiente(W_sgd, X_train_scaled, X_test_scaled, y_train, y_test, "SGD"))
resultados_finales.append(evaluar_gradiente(W_batch, X_train_scaled, X_test_scaled, y_train, y_test, "Batch Gradient Descent"))
resultados_finales.append(evaluar_gradiente(W_mini_batch, X_train_scaled, X_test_scaled, y_train, y_test, "Mini-Batch Gradient Descent"))
# Evaluación de regularización (Lasso, Ridge, ElasticNet)
for name, regressor in regresiones_lineales.items():
    regressor.fit(X_train_scaled, y_train)
    resultados_finales.append(evaluar_modelo(regressor, name, X_train_scaled, y_train, X_test_scaled, y_test))

# Mostrar los resultados finales
resultados_df = pd.DataFrame(resultados_finales)
resultados_df.head(10)


# Comparación de Códigos y Resultados Obtenidos

## Cambios en el código

### 1. Parámetros de Lasso y Ridge:

- **Lasso:**
  - En el primer código, el rango de `alpha` es más reducido `(np.logspace(-4, 4, 20))`, mientras que en el segundo código, se expandió para cubrir un rango más amplio `(np.logspace(-6, 2, 50))`, lo cual permite encontrar una mejor regularización en escenarios donde un ajuste fino es necesario.
  - Además, en el segundo código, se aumentó `max_iter` a `10,000` en Lasso, lo que permite más iteraciones y mayor tiempo para que el algoritmo converja en casos difíciles.

- **Ridge:**
  - En el primer código, el rango de `alpha` también es más estrecho `(np.logspace(-4, 4, 20))`, mientras que en el segundo se incrementa a un rango más amplio de `np.logspace(-6, 2, 50)`. Esto da mayor flexibilidad en la búsqueda del valor óptimo de regularización.

### 2. Parámetros de los Gradientes:

- **SGD:**
  - En la primera configuración: `epochs=1000` y `lr=0.001`.
  - En la segunda configuración: `epochs=15` y `lr=0.001`. La reducción significativa en el número de épocas muestra mejoras importantes en los resultados, sugiriendo que un menor número de iteraciones permitió evitar el sobreajuste.

- **Batch Gradient Descent:**
  - Primera configuración: `epochs=1000` y `lr=0.001`.
  - Segunda configuración: `epochs=650` y `lr=0.01`. Aumentar la tasa de aprendizaje junto con la reducción en las épocas mostró mejoras significativas tanto en el conjunto de entrenamiento como en el de prueba.

- **Mini-Batch Gradient Descent:**
  - Primera configuración: `epochs=100`, `lr=0.01`, y `batch_size=25`.
  - Segunda configuración: `epochs=1120`, `lr=0.01`, y `batch_size=240`. Aunque el número de épocas y el tamaño del batch cambiaron considerablemente, los resultados no muestran variaciones significativas entre ambas configuraciones.


## Comparación de Resultados

| Modelo                        | Train RMSE (Antes) | Train RMSE (Después) | Test RMSE (Antes) | Test RMSE (Después) | Train R2 (Antes) | Train R2 (Después) | Test R2 (Antes) | Test R2 (Después) |
|-------------------------------|--------------------|----------------------|-------------------|---------------------|------------------|--------------------|-----------------|-------------------|
| **SGD**                        | 5.681173           | 5.736999             | 6.460536           | 6.122241            | 0.634335          | 0.627114           | 0.563783         | 0.608271           |
| **Batch Gradient Descent**      | 7.976447           | 5.713968             | 7.588523           | 6.335249            | 0.279182          | 0.630101           | 0.398162         | 0.580538           |
| **Mini-Batch Gradient Descent** | 5.788892           | 5.666384             | 6.497669           | 6.411268            | 0.620337          | 0.636237           | 0.558754         | 0.570411           |
| **Lasso**                      | 5.664760           | 5.666052             | 6.357286           | 6.357096            | 0.636445          | 0.636279           | 0.577615         | 0.577640           |
| **Ridge**                      | 5.701377           | 5.697475             | 6.370033           | 6.361509            | 0.631730          | 0.632234           | 0.577156         | 0.577053           |

## Evaluación de los Resultados:

- **SGD:**
  Se observa un pequeño aumento en el **Train RMSE** (de 5.68 a 5.74), pero el **Test RMSE** disminuyó de 6.46 a 6.12, lo que indica que el modelo mejoró su rendimiento en los datos de prueba después del ajuste de hiperparámetros. El **Test R2** también subió de 0.56 a 0.61, lo que indica que el modelo es capaz de explicar mejor la variabilidad de los datos de prueba tras el ajuste.

- **Batch Gradient Descent:**
  El **Train RMSE** mejoró considerablemente (de 7.98 a 5.71), lo que indica que el modelo se ajustó mucho mejor a los datos de entrenamiento. Además, el **Test RMSE** bajó de 7.59 a 6.33, lo que sugiere que los ajustes de hiperparámetros tuvieron un impacto positivo, mejorando su capacidad para generalizar en los datos de prueba. El **Test R2** también mejoró de 0.40 a 0.58.

- **Mini-Batch Gradient Descent:**
  Las métricas también mejoraron tras el ajuste, aunque de manera menos dramática en comparación con otros modelos. El **Test RMSE** bajó ligeramente de 6.50 a 6.41, y el **Test R2** subió de 0.56 a 0.57. Estos cambios indican una mejora leve en el rendimiento general del modelo.

- **Lasso:**
  Las métricas para el modelo **Lasso** no cambiaron significativamente después del ajuste de hiperparámetros. El **Test RMSE** se mantuvo prácticamente igual (6.357 frente a 6.357) y el **Test R2** también permaneció casi idéntico (0.5776). Esto sugiere que los ajustes en los hiperparámetros no tuvieron un efecto notable en el rendimiento del modelo.

- **Ridge:**
  El modelo **Ridge** mostró una leve mejoría. El **Train RMSE** disminuyó de 5.70 a 5.69, mientras que el **Test RMSE** se mantuvo prácticamente igual (6.37 vs. 6.36). El **Test R2** también se mantuvo casi sin cambios. Esto indica que los ajustes de hiperparámetros tuvieron un impacto mínimo en el rendimiento del modelo.


## Evaluación Global:

Los cambios en los hiperparámetros tuvieron un impacto positivo más significativo en los modelos **SGD** y **Batch Gradient Descent**, con mejoras tanto en **Test RMSE** como en **Test R2**. **Mini-Batch Gradient Descent** también mostró una ligera mejora. Por otro lado, los modelos regularizados como **Lasso** y **Ridge** apenas mostraron cambios, lo que sugiere que las modificaciones en los hiperparámetros no fueron tan efectivas en estos casos.

### Conclusión:
- **SGD** y **Batch Gradient Descent** fueron los que más se beneficiaron de los ajustes, con mejoras notables en su capacidad de generalización.
- **Lasso** y **Ridge** no mostraron grandes variaciones tras el ajuste de hiperparámetros, por lo que es probable que estos modelos ya estuvieran bien ajustados o que no sean tan sensibles a los cambios realizados.

# Comparación de Modelos

Para determinar cuál es el mejor modelo de regresión, usaremos como métrica principal el **RMSE** (Root Mean Squared Error) en el conjunto de **prueba**, ya que mide la magnitud de los errores en la predicción y nos permite identificar qué tan bien el modelo generaliza en datos no vistos. También consideraremos el **R²**, que nos indica qué proporción de la variabilidad de la variable dependiente es explicada por el modelo.

#### Resumen de las métricas clave:
- **Regresión Lineal**:
  - Test RMSE: 6.36
  - Test R²: 0.58
- **SGD**:
  - Test RMSE: 6.12
  - Test R²: 0.61
- **Batch Gradient Descent**:
  - Test RMSE: 6.34
  - Test R²: 0.58
- **Mini-Batch Gradient Descent**:
  - Test RMSE: 6.41
  - Test R²: 0.57
- **Lasso**:
  - Test RMSE: 6.36
  - Test R²: 0.58
- **Ridge**:
  - Test RMSE: 6.36
  - Test R²: 0.58
- **ElasticNet**:
  - Test RMSE: 6.36
  - Test R²: 0.58


#### Análisis:

1. **SGD** tiene el mejor **RMSE** en el conjunto de prueba con un valor de **6.12**, lo que indica que tiene el menor error de predicción entre todos los modelos evaluados. Además, su **R²** de **0.61** es el más alto, lo que significa que explica el 61% de la variabilidad en los datos de prueba, lo que sugiere un excelente ajuste y capacidad de generalización.

2. **Batch Gradient Descent** tiene un **RMSE** cercano de **6.34** con un **R²** de **0.58**. Aunque su error es ligeramente mayor que el de SGD, sigue mostrando un rendimiento muy competitivo y una buena capacidad para generalizar.

3. **Mini-Batch Gradient Descent** tiene un **RMSE** de **6.41** y un **R²** de **0.57**. Aunque su rendimiento es ligeramente inferior a los métodos anteriores, sigue siendo competitivo con un error relativamente bajo y una buena capacidad de ajuste.

4. Los modelos más tradicionales como la **Regresión Lineal**, **Lasso**, **Ridge** y **ElasticNet** tienen todos un **RMSE** de **6.36** y un **R²** de **0.58**. Esto indica que estos modelos no generalizan tan bien como los métodos de gradiente. Si bien explican una parte importante de la variabilidad de los datos (con un **R²** de 0.58), el error absoluto que presentan es mayor en comparación con SGD y Batch Gradient Descent.

#### Conclusión:

El modelo **SGD (Stochastic Gradient Descent)** se destaca como el mejor modelo en este conjunto de datos, con el menor **Test RMSE** de **6.12** y el mayor **R²** de **0.61**. Esto lo convierte en la opción más adecuada, ya que presenta el mejor equilibrio entre precisión y capacidad de generalización. **Batch Gradient Descent** también tiene un buen rendimiento, con un RMSE de **6.34**, lo que lo hace una alternativa viable. Por otro lado, los modelos tradicionales, como la **Regresión Lineal**, **Lasso**, **Ridge**, y **ElasticNet**, presentan un rendimiento inferior, aunque sus **R²** son similares, indicando que tienen más dificultades para reducir los errores absolutos.


# Conclusión final del trabajo

Este proyecto de análisis de precios de casas se centró en la preparación de datos, manejo de valores atípicos, imputación de valores faltantes y comparación de múltiples enfoques de regresión para predecir el valor de una vivienda (variable objetivo "MEDV").

1. **Preprocesamiento de datos**: 
   - La eliminación de filas con un alto número de valores faltantes,esto se hizo para garantizar la calidad de los datos. Se implementó un método de imputación KNN para manejar valores faltantes, lo que ayudó a preservar las observaciones al completar las características faltantes con datos similares.

2. **Análisis exploratorio**:
   - Los análisis gráficos como los boxplots y pairplots permitieron identificar la distribución de las variables y observar posibles correlaciones entre ellas. Este tipo de visualización fue útil para detectar posibles relaciones entre las variables explicativas y la variable objetivo, lo cual guió la elección de los modelos.

3. **Modelado y evaluación**:
   - Se implementaron múltiples técnicas de regresión lineal regularizada, como Ridge, Lasso y ElasticNet, junto con modelos de gradiente descendente (GD, SGD, y Mini-Batch GD). Estos modelos se ajustaron usando datos escalados para mejorar la convergencia y la estabilidad de los coeficientes del modelo.
   - Los resultados se evaluaron utilizando métricas como el MSE (Error Cuadrático Medio), MAE (Error Absoluto Medio), y MAPE (Porcentaje de Error Absoluto Medio), lo que permitió comparar el rendimiento entre los distintos enfoques.
   - Se implementaron gráficos para visualizar los errores de entrenamiento y prueba a lo largo de las iteraciones, lo que permitió un monitoreo visual de la optimización del modelo.


### Reflexiones finales:
A lo largo de este análisis, pudimos ver cómo diferentes técnicas de modelado pueden abordar con éxito problemas de regresión, siempre y cuando se sigan bien los pasos previos como el preprocesamiento y la selección adecuada de modelos. La combinación de modelos como Lasso y Ridge, junto con técnicas de gradiente descendente, nos ofreció una perspectiva clara de los enfoques lineales para hacer predicciones.

En resumen, al combinar métodos como la imputación de datos, la detección de valores atípicos y el uso de varios modelos de regresión, logramos un análisis más sólido que mejora la precisión en la predicción de precios de viviendas.